In [14]:
import torch
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from functools import reduce
from torchvision import transforms
from torch import optim, nn

In [15]:
with open('lotr.txt', 'r') as file:
    text = file.read()

text = text.lower()
text = text.replace('#', '')
text = text.replace('*', '')
text = text.replace('(', '')
text = text.replace(')', '')
text = text.replace('`', "'")
text = text.replace(')', '')
text = text.replace('–', ' ')
text = text.replace('-', ' ')
text = text.replace('—', ' ')
text = text.replace('»', '"')
text = text.replace('«', '"')
text = text.replace('_', ' ')
text = text.replace('’', "'")
text = text.replace('‘', "'")
text = text.replace('ó', 'o')
text = text.replace('{', '')
text = text.replace('}', '')
text = text.replace('µ', ' ')
text = text.replace('¤', '')
text = text.replace('¢', '')
text = text.replace('¢', '')
text = text.replace('®', '')
text = text.replace('¥', '')
text = text.replace('<br>', '')
text = text.replace('<h4>', '')
text = text.replace('</h4>', '')
text = text.replace('/', '')
text = text.replace('&', 'e')
text = text.replace('=', 'o')
text = text.replace('‚', ',')

emb = pd.read_csv(r'glove.6B\glove.6B.50d.txt', sep = ' ', quotechar=None, quoting=3, header=None)
emb.index = emb.iloc[:, 0]
emb.drop(columns=emb.columns[0], inplace=True)
corpus = set(word for word in text.split())
word_in_corpus = [i for i in emb.index if i in corpus]
emb = emb.loc[word_in_corpus, :]
emb = pd.DataFrame(np.round(emb.values, 4), index=emb.index)
emb = emb.apply(lambda x: x/np.linalg.norm(x), axis=1)
emb.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,,,,,,,,,,,,,,,,,,,,,
the,0.084141,0.050263,-0.083014,0.024498,0.069507,-0.008958,-0.100023,-0.035951,-0.000141,-0.132170,...,-0.060127,-0.031704,-0.069970,-0.009179,-0.089073,0.037803,0.000564,-0.037058,-0.023169,-0.158178
",",0.002993,0.052883,-0.037741,0.091450,0.142502,0.106547,-0.095693,-0.124256,-0.081289,-0.053463,...,-0.017933,0.140693,0.071709,-0.104425,0.050895,0.080463,-0.084460,-0.126534,0.009982,0.067867
.,0.034092,0.067870,-0.037690,0.039759,0.071333,0.076393,-0.097779,-0.069916,-0.101197,-0.066318,...,-0.000022,0.015517,0.019767,-0.023118,-0.031326,0.050171,-0.018171,-0.080171,0.003688,0.022983
of,0.142955,0.115191,-0.095155,0.036420,0.109865,0.146486,0.036642,-0.105708,0.020944,-0.035451,...,-0.070075,0.057465,0.015274,-0.012550,-0.078671,0.046206,-0.043623,-0.045520,-0.018946,-0.162184
to,0.137284,-0.007928,0.060905,-0.035889,0.086667,0.006496,-0.083480,0.026690,-0.060219,-0.017208,...,-0.019044,0.003692,0.042466,-0.006234,-0.039783,0.016603,-0.019024,-0.014788,-0.013053,-0.052533


In [16]:
emb.to_csv(r'pre_trained_model\\embedding_matrix.csv')

In [17]:
emb.to_json('glove_lotr.json')

In [18]:
emb = pd.read_csv(r'glove.6B\glove.6B.200d.txt', sep = ' ', quotechar=None, quoting=3, header=None)
emb.index = emb.iloc[:, 0]
emb.drop(columns=emb.columns[0], inplace=True)
corpus = set(word for word in text.split())
word_in_corpus = [i for i in emb.index if i in corpus]
emb = emb.loc[word_in_corpus, :]
emb = pd.DataFrame(np.round(emb.values, 4), index=emb.index)
emb = emb.apply(lambda x: x/np.linalg.norm(x), axis=1)
emb.to_json('glove_lotr_200d.json')
emb.head()

FileNotFoundError: File b'glove.6B\\glove.6B.200d.txt' does not exist

In [ ]:
# -*- coding: utf-8 -*-
class LOTRDataset(Dataset):
    
    def __init__(self, filepath, emb, transform=None):
        
        ### Load data
        text = open(filepath, 'r', encoding='utf-8-sig').read()
        
        ### Preprocess data
        # Remove the first and the last part (which are not sonnets in the text file)
        # Lower case
        text = text.lower()
        text = text.replace('#', '')
        text = text.replace('*', '')
        text = text.replace('(', '')
        text = text.replace(')', '')
        text = text.replace('`', "'")
        text = text.replace(')', '')
        text = text.replace('–', ' ')
        text = text.replace('-', ' ')
        text = text.replace('—', ' ')
        text = text.replace('»', '"')
        text = text.replace('«', '"')
        text = text.replace('_', ' ')
        text = text.replace('’', "'")
        text = text.replace('‘', "'")
        text = text.replace('ó', 'o')
        text = text.replace('{', '')
        text = text.replace('}', '')
        text = text.replace('µ', ' ')
        text = text.replace('¤', '')
        text = text.replace('¢', '')
        text = text.replace('¢', '')
        text = text.replace('®', '')
        text = text.replace('¥', '')
        text = text.replace('<br>', '')
        text = text.replace('<h4>', '')
        text = text.replace('</h4>', '')
        text = text.replace('/', '')
        text = text.replace('&', 'e')
        text = text.replace('=', 'o')
        text = text.replace('‚', ',')
        

        # Extract the sonnets (divided by empty lines and roman numerals)
        sentences = re.split('[.]', text)
        sentences = [i for i in sentences if len(i.split()) > 16]
        chapter_list = sentences
        ### Char to number
        char_to_number = {key: value for key, value in zip(emb.index, emb.values)}
                
        ### Store data
        self.corpus = text
        self.chapter_list = chapter_list
        self.transform = transform
        self.emb = emb
        self.char_to_number = char_to_number
        #self.number_to_char = number_to_char
        
    def __len__(self):
        return len(self.chapter_list)
        
    def __getitem__(self, idx):
        # Get sonnet text
        text = self.chapter_list[idx]
        """
        if len(text.split()) < 9:
            print(self.chapter_list[idx])
            print(text)
        """
        # Encode with numbers
        encoded = encode_text(self.char_to_number, text, self.emb)
        # Create sample
        sample = {'text': text, 'encoded': encoded}
        # Transform (if defined)
        if self.transform:
            sample = self.transform(sample)
        return sample


def encode_text(char_to_number, text, emb):
    encoded = [char_to_number[c] for c in re.findall(r"[\w']+|[.,!?;]", text) if c in emb.index]
    return encoded


def decode_text(emb, encoded):
    text = [emb.index[(emb == c).all(axis=1)][0] for c in encoded]
    #text = [number_to_char[c] for c in encoded]
    #text = reduce(lambda s1, s2: s1 + s2, text)
    return text


class RandomCrop():
    
    def __init__(self, crop_len):
        self.crop_len = crop_len
        
    def __call__(self, sample):
        text = sample['text']
        encoded = sample['encoded']
        # Randomly choose an index
        tot_words = len(text.split())
        #start_words = np.random.randint(0, tot_words - self.crop_len)
        start_words = 0
        end_words = start_words + self.crop_len
        new_text = ' '.join(text.split()[start_words:end_words])
        #print(len(text.split()))
        if len(new_text.split()) < self.crop_len:
            print(len(new_text.split()))
            print(new_text.split())
        return {**sample,
                'text': new_text,
                'encoded': encoded[start_words: end_words]}
    


class OneHotEncoder():
    
    def __init__(self, emb=None):
        self.emb = emb
        #self.alphabet_len = alphabet_len
        
    def __call__(self, sample):
        # Load encoded text with numbers
        encoded = np.array(sample['encoded'])
        # Create one hot matrix
        #encoded_onehot = create_one_hot_matrix(encoded, self.alphabet_len)
        encoded_onehot = encoded
        return {**sample,
                'encoded_onehot': encoded_onehot}
        
                
class ToTensor():
    
    def __call__(self, sample):
        # Convert one hot encoded text to pytorch tensor
        encoded_onehot = torch.tensor(sample['encoded_onehot'])
        return {'encoded_onehot': encoded_onehot}

In [ ]:
#if __name__ == '__main__':
#%% Initialize dataset
filepath = 'lotr.txt'

dataset = LOTRDataset(filepath, emb)

#%% Test sampling
sample = dataset[0]
print('##############')
print('##############')
print('TEXT')
print('##############')
print(sample['text'])

print('##############')
print('##############')
print('ENCODED')
print('##############')
print(sample['encoded'][0])

#%% Test decode function
encoded_text = sample['encoded'] #A list of np.array (,50)
decoded_text = decode_text(dataset.emb, encoded_text)

print('##############')
print('##############')
print('DECODED')
print('##############')
print(decoded_text)

In [ ]:
crop_len = 10
rc = RandomCrop(crop_len)
sample = rc(sample)
print('Cropped text: ', sample['text'])
print('Cropeed decoded text', decode_text(dataset.emb, sample['encoded']))

In [ ]:
#%% Test OneHotEncoder
#alphabet_len = len(dataset.alphabet)
ohe = OneHotEncoder()
sample = ohe(sample)
print(sample['encoded_onehot'].shape)
print('Cropped text: ', sample['text'])
print('Cropeed decoded text', decode_text(dataset.emb, sample['encoded_onehot']))

In [ ]:
#%% Test ToTensor
print(sample['encoded'][0][0])
tt = ToTensor()
sampler = tt(sample)
print(sampler['encoded_onehot'][0][0])
print('Cropeed decoded text', decode_text(dataset.emb, sampler['encoded_onehot'].numpy()))

In [ ]:
#%% Test dataloader
filepath = 'lotr.txt'
crop_len = 7
#alphabet_len = len(dataset.alphabet)
trans = transforms.Compose([RandomCrop(crop_len),
                            OneHotEncoder(),
                            ToTensor()
                            ])
dataset = LOTRDataset(filepath, emb, transform=trans)
dataloader = DataLoader(dataset, batch_size=len(dataset.chapter_list)//80, shuffle=True)

for batch_sample in dataloader:
    batch_onehot = batch_sample['encoded_onehot']
    print(batch_onehot.shape)
    
validation = batch_onehot

In [ ]:
validation = batch_onehot

In [ ]:
# -*- coding: utf-8 -*-
class Network(nn.Module):
    
    def __init__(self, input_size, hidden_units, layers_num, dropout_prob=0):
        # Call the parent init function (required!)
        super().__init__()
        # Define recurrent layer
        self.rnn = nn.LSTM(input_size=input_size, 
                           hidden_size=hidden_units,
                           num_layers=layers_num,
                           dropout=dropout_prob,
                           batch_first=True)
        # Define output layer
        self.out = nn.Linear(hidden_units, input_size)
        
    def forward(self, x, state=None):
        # LSTM
        x, rnn_state = self.rnn(x, state)
        # Linear layer
        x = self.out(x)
        return x, rnn_state

In [ ]:
def train_batch(net, batch_onehot, loss_fn, optimizer):
    
    batch_onehot = batch_onehot.float()
    ### Prepare network input and labels
    # Get the labels (the last word of each sequence)
    labels_onehot = batch_onehot[:, -1, :]
    #labels_numbers = labels_onehot.argmax(dim=1)
    # Remove the labels from the input tensor
    net_input = batch_onehot[:, :-1, :]
    # batch_onehot.shape =   [50, 100, 38]
    # labels_onehot.shape =  [50, 38]
    # labels_numbers.shape = [50]
    # net_input.shape =      [50, 99, 38]
    
    ### Forward pass
    # Eventually clear previous recorded gradients
    optimizer.zero_grad()
    # Forward pass
    net_out, _ = net(net_input)
    #net_out  = nn.functional.softmax(net_out, dim=1)
    #print(net_out.shape)
    ### Update network
    #print('Shape of out and label: {}, {}'.format(net_out[:, -1, :].shape, labels_onehot.shape))
    my_loss = torch.mean(torch.norm(net_out[:, -1, :] - labels_onehot, dim = 1))
    
    #print('My loss: {}'.format(my_loss))
    #print('With shape: {}'.format(my_loss.shape))
    # Evaluate loss only for last output
    loss = loss_fn(net_out[:, -1, :], labels_onehot)
    #print("It's loss: {}".format(loss))
    # Backward pass
    #loss.backward(retain_graph=True)
    loss.backward()
    # Update
    optimizer.step()
    # Return average batch loss
    return float(loss.data), my_loss, net_out, labels_onehot

In [ ]:
def customized_loss(y_true, y_pred):
    loss = torch.mean(torch.norm(y_true-y_pred, dim=1))
    return loss

In [ ]:
def testing(net, embedding_matrix, dataset, sentences='he went', length=30):
    state = sentences
    X = embedding_matrix.values


    with torch.no_grad():
        # Encode seed
        seed_encoded = encode_text(dataset.char_to_number, state.lower(), embedding_matrix)
        # One hot matrix
        #seed_onehot = create_one_hot_matrix(seed_encoded, 47)
        # To tensor
        seed_onehot = torch.tensor(seed_encoded).float()
        #print(seed_onehot)
        # Add batch axis
        seed_onehot = seed_onehot.unsqueeze(0)
        # Forward pass
        seed_onehot = seed_onehot.to(device)
        net_out, net_state = net(seed_onehot)
        # Get the most probable last output index
        next_word_encoded = net_out[:, -1, :]
        closest_value = np.linalg.norm((X - next_word_encoded.to('cpu').numpy()[0]), axis = 1)
        closest_word = embedding_matrix.index[np.argmin(closest_value)]
        state += ' ' + closest_word
        # Print the seed letters
        print(state, end=' ', flush=True)
    #%% Generate sonnet
    new_line_count = 0
    tot_char_count = 0
    while True:
        with torch.no_grad(): # No need to track the gradients
            # The new network input is the one hot encoding of the last chosen letter
            net_input = encode_text(dataset.char_to_number, state.lower(), embedding_matrix)
            net_input = torch.reshape(torch.tensor(net_input).float(), (1, -1, 50))
            #print(net_input.shape)
            #net_input = net_input.unsqueeze(0)
            # Forward pass
            net_input = net_input.to(device)
            net_out, net_state = net(net_input, net_state)
            
            # Get the most probable letter index
            np_out = net_out[:, -1, :].to('cpu').numpy()[0]
            np_out = np_out/np.linalg.norm(np_out)
            closest_value = np.linalg.norm((X - np_out), axis = 1)
            closest_word = embedding_matrix.index[np.argmin(closest_value)]
            
            state += ' ' + closest_word
            end = False
            if closest_word == '.':
                print(closest_word, end='\n', flush=True)
                end = True
            else:
                if end:
                    print(closest_word.upper(), end=' ', flush=True)
                else:
                    print(closest_word, end=' ', flush=True)
                end = False
            # Count total letters
            tot_char_count += 1
            # Count new lines
            # Break if 14 lines or 2000 letters
            if tot_char_count > length:
                break

In [19]:
import time

#%% Check device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print('Selected device:', device)

#%% Initialize network
net = Network(input_size=50, 
              hidden_units=256, 
              layers_num=2, 
              dropout_prob=0.3)
net.to(device)
validation = validation.to(device).float()
#%% Train network

# Define optimizer
optimizer = optim.Adam(net.parameters(), lr = 0.003)
# Define loss function
loss_fn = nn.MSELoss()

tr_loss, val_loss = [], []
ver = 20
# Start training
for epoch in range(600):
    start = time.time()
    if epoch%ver == 0:
        if len(loss) > ver:
            print('\n\tAvarage lost in the last {}: {}\n'.format(ver, np.mean(loss[epoch-ver:epoch])))
            testing(net, emb, dataset, 'does he', length = 50)
        print('##################################')
        print('## EPOCH %d' % (epoch + 1))
        print('##################################')
    b_losses, m_losses = [], []
    # Iterate batches
    for batch_sample in dataloader:
        # Extract batch
        batch_onehot = batch_sample['encoded_onehot'].to(device)
        
        # Update network
        batch_loss, my_loss, out, y_true = train_batch(net, batch_onehot, loss_fn, optimizer)    
        
        b_losses.append(batch_loss)
        m_losses.append(my_loss)
        
        
        with torch.no_grad():
            y_validation = validation[:, -1, :]
            #labels_numbers = labels_onehot.argmax(dim=1)
            # Remove the labels from the input tensor
            val_input = validation[:, :-1, :]
            validation_pred, _ = net(val_input)
            #net_out  = nn.functional.softmax(net_out, dim=1)
            #print(net_out.shape)
            ### Update network
            # Evaluate loss only for last output
            loss_val = loss_fn(validation_pred[:, -1, :], y_validation)
        
        #Clear cache
        del batch_onehot
        torch.cuda.empty_cache()
        #if epoch%20 == 0:
            #print('\t Training loss (single batch):', batch_loss)
    loss.append(torch.mean(torch.tensor(b_losses)))
    #if epoch%20 == 0:
    print('MSE loss: {}.'.format(torch.mean(torch.tensor(b_losses))), end='\t')
    print('Validation loss: {}'.format(loss_val.data), end='\t')
    print('Time: {}'.format(np.round(time.time()-start), 4))
print(loss[-1])

Selected device: cuda


NameError: name 'loss' is not defined

In [ ]:
def normalize(batch_out):
    batch_size, nfeatures = batch_out.shape[0], batch_out.shape[-1] 
    batch = torch.reshape(batch_out, (-1, nfeatures))
    batch = batch/torch.norm(batch, dim=1).view(batch.shape[0], 1)
    return torch.reshape(batch_out, (batch_size, -1, nfeatures))

out = normalize(out)

In [21]:
state = 'he went up and down the shire'
X = emb.values


with torch.no_grad():
    # Encode seed
    seed_encoded = encode_text(dataset.char_to_number, state.lower(), emb)
    # One hot matrix
    #seed_onehot = create_one_hot_matrix(seed_encoded, 47)
    # To tensor
    seed_onehot = torch.tensor(seed_encoded).float()
    #print(seed_onehot)
    # Add batch axis
    seed_onehot = seed_onehot.unsqueeze(0)
    # Forward pass
    seed_onehot = seed_onehot.to(device)
    net_out, net_state = net1(seed_onehot)
    # Get the most probable last output index
    next_word_encoded = net_out[:, -1, :]
    closest_value = np.linalg.norm((X - next_word_encoded.to('cpu').numpy()[0]), axis = 1)
    closest_word = emb.index[np.argmin(closest_value)]
    state += ' ' + closest_word
    # Print the seed letters
    print(state, end=' ', flush=True)
#%% Generate sonnet
new_line_count = 0
tot_char_count = 0
while True:
    with torch.no_grad(): # No need to track the gradients
        # The new network input is the one hot encoding of the last chosen letter
        net_input = encode_text(dataset.char_to_number, state.lower(), emb)
        net_input = torch.reshape(torch.tensor(net_input).float(), (1, -1, 50))
        #print(net_input.shape)
        #net_input = net_input.unsqueeze(0)
        # Forward pass
        net_input = net_input.to(device)
        net_out, net_state = net1(net_input, net_state)
        
        # Get the most probable letter index
        closest_value = np.linalg.norm((X - net_out[:, -1, :].to('cpu').numpy()[0]), axis = 1)
        closest_word = emb.index[np.argmin(closest_value)]
        state += ' ' + closest_word
        print(closest_word, end=' ', flush=True)
        # Count total letters
        tot_char_count += 1
        # Count new lines
        # Break if 14 lines or 2000 letters
        if tot_char_count > 100:
            break

he went up and down the shire and went still well be as green as look some more well for only . his her him , as n't they heard daddy . just came just out . still apart in the twigs and coming up keep put with with flesh like they more instead well well him but but come did ? but but it the though but though but watching just instead cursed was pretended . even come only i him him some leaving to on when , they . but bouncing push the hand which one in the with of the demons took went came to but 

In [17]:
torch.save(net, 'model_at_03_dec')

C:\Users\fgrim\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [28]:
device = torch.device('cuda')
net1 = torch.load(r'model_long')
print(net1.rnn)
testing(net1, emb, dataset, 'he went up and down the shire', length = 50)

LSTM(47, 512, num_layers=2, batch_first=True, dropout=0.1)


RuntimeError: input.size(-1) must be equal to input_size. Expected 47, got 50

In [19]:
for batch_sample in dataloader:
    # Extract batch
    batch_onehot = batch_sample['encoded_onehot'][0:5, :, :]
    for i in range(5):
        print(decode_text(dataset.emb, batch_onehot[i, :, :].numpy()))
    
    batch_onehot_dev = batch_onehot.to(device)
    # Update network
    batch_loss, my_loss, out, y_true = train_batch(net, batch_onehot_dev, loss_fn, optimizer)
    print(out.shape)
    
    values = []
    for i in range(5):
        print('##############')
        out[i, :, :].shape
        out_word = out[i, :, :].cpu().detach().numpy()
        for j in range(6):
            values.append(out_word[j])
            #Letter to predict
            true_word = decode_text(dataset.emb, batch_onehot[i, :, :].numpy())[j+1]
            true_value = np.linalg.norm(batch_onehot[i, :, :].numpy()[j+1] - out_word[j])
            true_value = float(torch.norm(batch_onehot[i, j+1, :].cpu().float() - out[i, j, :].cpu()).data)
            print(true_word, end = '\t')
            closest_value = np.argmin(np.linalg.norm((X - out_word[j]), axis = 1))
            #print(np.linalg.norm((X - out_word[j]), axis = 1).shape)
            closest_word = emb.index[closest_value]
            print(closest_word)
            print(true_value, '\t', np.min((np.linalg.norm((X - out_word[j]), axis = 1))))
            #print(np.min((np.linalg.norm((X - out_word[j]), axis = 1)))-true_value)
            
            #print(emb[emb.index == ''])
    
    b_losses.append(batch_loss)
    break

['he', 'had', 'not', 'forgotten', 'the', 'look', 'of']
['was', 'very', 'anxious', 'to', 'find', 'some', 'sign']
['when', 'autumn', 'came', ',', 'he', 'knew', 'that']
['i', 'imagine', 'what', 'spring', 'would', 'look', 'like']
["'", 'when', 'he', 'heard', 'what', 'frodo', 'had']
torch.Size([5, 6, 50])
##############
had	when
0.40604567527770996 	 0.3945769998408468
not	turned
0.8561952114105225 	 0.43178995707111495
forgotten	occasion
1.2295607328414917 	 0.8516743971424025
the	confirm
1.3213441371917725 	 0.9896899455375435
look	ground
1.0786402225494385 	 0.8247855164957774
of	of
0.03832031413912773 	 0.038320315528186946
##############
very	still
0.5128685235977173 	 0.32169898089498167
anxious	made
0.9829562902450562 	 0.5219056313426617
to	,
0.5812139511108398 	 0.23428700202510533
find	pity
0.887028694152832 	 0.7011574674714168
some	him
0.7747485041618347 	 0.33100224672743744
sign	sign
0.46773993968963623 	 0.4677399386228759
##############
autumn	but
1.002932071685791 	 0.24991

In [ ]:
print(np.shape(values))
df = pd.DataFrame(values)
df.head(40)

In [ ]:
emb.loc['banners', :]